In [1]:
import sys
from PySide6 import QtWidgets
from PySide6 import QtCore
from PySide6 import QtGui
from PySide6.QtGui import *
from PySide6.QtWidgets import *
from PySide6.QtCore import *
import pyqtgraph as pg
import numpy as np
import pandas as pd
from time import *
import concurrent.futures
import os
import cv2
import traceback
import ctypes
import warnings
import traceback
from collections import defaultdict
import inspect
myappid = 'nil.npm.pyqt.2' # arbitrary string
ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)

from importlib import reload

import ui_main
reload(ui_main)
from ui_main import *

import npmgraph
reload(npmgraph)
from npmgraph import *

from scaleBar import scaleBar

videoFormats = [
    "avi",
    "mp4",
    "mov",
    "mkv",
    "wmv",
    "flv",
    "mpeg",
    "mpg",
]

data=None
videoData=None

def getMaxThreads():
    # Get the number of available CPUs
    numCPUs = os.cpu_count()
    
    # Default maximum threads in ThreadPoolExecutor
    maxThreads = numCPUs * 5 if numCPUs else 1
    
    return maxThreads

def resizeVideos(rows, cols, width, height, layout): # resize all videos to fit
    videos=[]
    for row in range(rows): 
        for col in range(cols):
            widget = layout.itemAtPosition(row, col)
            if widget is not None:
                video = widget.widget()
                video.setMaximumSize(width, height)
                video.setIconSize(QtCore.QSize(width, height))
                video.setStyleSheet("border: none; background: transparent;")
                videos.append(video)
    return videos

def clearVideos(layout):
    for i in reversed(range(layout.count())):
        widget = layout.itemAt(i).widget()
        if widget is not None:
            widget.setParent(None)

def cv2FrameToPixmap(frame):
    # Convert BGR (OpenCV format) to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Convert the image to a QImage
    height, width, channel = frame.shape
    bytesPerLine = 3 * width
    qImage = QImage(frame.data, width, height, bytesPerLine, QImage.Format_RGB888)

    # Convert QImage to QPixmap
    pixmap = QPixmap.fromImage(qImage)
    return pixmap

class WorkerSignals(QObject): # Source: https://www.pythonguis.com/tutorials/multithreading-pyside6-applications-qthreadpool/
    '''
    Defines the signals available from a running worker thread.
    
    Supported signals are:
    
    error
        tuple (exctype, value, traceback.format_exc() )

    result
        object data returned from processing, anything
    '''

    error = Signal(tuple)
    result = Signal(object)
    
class VideoWorker(QRunnable): # Source: https://www.pythonguis.com/tutorials/multithreading-pyside6-applications-qthreadpool/
    def __init__(self, mutex, fn, *args, **kwargs):
        super(VideoWorker, self).__init__()

        # Store constructor arguments (re-used for processing)
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()
        self.mutex = mutex

        # Add the callback to our kwargs
        self.kwargs['resultCallback'] = self.signals.result

        # run flag
        self._running = True

    def stop(self):
        print('STOP')
        self.running = False # stop flag
    
    @Slot()
    def run(self):
        '''
        Initialise the runner function with passed args, kwargs.
        '''
        if self._running:
            # Retrieve args/kwargs here; and fire processing using them
            try:
                result = self.fn(*self.args, **self.kwargs)
            except:
                traceback.print_exc()
                exctype, value = sys.exc_info()[:2]
                self.signals.error.emit((exctype, value, traceback.format_exc()))
            else:
                self.mutex.lock()
                try:
                    self.signals.result.emit(result)  # Return the result of the processing
                finally:
                    self.mutex.unlock()       
                    
class Dialog(QDialog, Ui_Dialog): # save position dialog box
    def __init__(self, data, parent=None):
        QDialog.__init__(self, parent)
        self.data = data
        self.setupUi(self)
        self.setWindowTitle("NPM Graph")
        self.setWindowIcon(QtGui.QIcon('icon.png'))
        self.npmPlot.showGrid(x=True, y=True)
        self.npmPlot.setLabel("left", "Velocity (um/s)")
        self.npmPlot.setLabel("bottom", "Voltage (V)")
        self.npmPlot.setLimits(xMin=-5, xMax=5, yMin=-5, yMax=5)
        self.npmPlot.setXRange(-5, 5)
        self.npmPlot.setYRange(-5, 5)
        self.curve = self.npmPlot.plot([0], [0], pen='r', symbol=None, name='Mean Velocity')
        self.errorBars = pg.ErrorBarItem()
        self.npmPlot.addItem(self.errorBars)

    def updatePlot(self, data):
        if data:
            try:
                voltage, velocity, error = zip(*data) # unpack data
                self.curve.setData(voltage, velocity) # update data points
                
                self.errorBars.setData(x=np.array(voltage), # update the error bars
                                        y=np.array(velocity), 
                                        top=np.array(error), 
                                        bottom=np.array(error),
                                        beam=0.5)
            except Exception as e:
                err = traceback.format_exc()
                print(err)
        
class MainWindow(QtWidgets.QMainWindow, Ui_MainWindow): # main window
    def __init__(self):
        super(MainWindow, self).__init__()
        self.initialized = False
        self.setupUi(self)
        self.mutex = QMutex() # locking threads
        self.rectBufferMutex = QMutex() # locking threads
        
        # setup threads
        self.threadPool = QtCore.QThreadPool()
        self.threadPool.setMaxThreadCount(getMaxThreads())
        self.videoConnections = []
        
        # set icon and title
        self.setWindowIcon(QtGui.QIcon('icon.png'))
        self.setWindowTitle("NPM Video Analyzer")

        self.videoFiles = [] # store file paths
        
        self.actionOpen_Video.triggered.connect(self.openFile)  # Menubar "Open"
        self.actionSave.triggered.connect(self.exportVideo) # Menubar "Export"
        self.actionClear.triggered.connect(self.onClear) # Menubar "Clear"
        self.actionScale_Bar.triggered.connect(self.onScaleBar) # Menubar "Scale Bar"
        self.actionPoints_2.triggered.connect(self.onPoints) # Menubar "Points"
        self.actionBounding_Boxes_2.triggered.connect(self.onBB) # Menubar "Bounding Boxes"
        self.actionCollate_Data_NPM.triggered.connect(self.onCollateData) # Menubar "Collate by Voltage (NPM Only)"
        self.NPMPlot = Dialog(None)
        self.scaleBarOn = 1

        # connect sliders
        self.contourValue.valueChanged.connect(self.contourFunction)
        self.contrastValue.valueChanged.connect(self.contrastFunction)
        self.thresholdValue.valueChanged.connect(self.thresholdFunction)
        
        self.frameDiffValue.valueChanged.connect(self.onFrameDifferencing)
        self.frameDiffToggle.toggled.connect(self.onFrameDifferencing)
        
        self.subBackValue.valueChanged.connect(self.subtractBackgroundFunction)
        self.subBackToggle.toggled.connect(self.subtractBackgroundFunction)
        self.subBackMethod.currentIndexChanged.connect(self.subtractBackgroundFunction)
        
        self.consecutiveFramesValue.valueChanged.connect(self.onConsecutiveFramesValue)
        self.record.toggled.connect(self.onRecord)
        self.contDetectSlider.valueChanged.connect(self.contDetectValue.setValue)
        self.contDetectValue.valueChanged.connect(self.contDetectSlider.setValue)
        self.contDetectValue.valueChanged.connect(self.onPersistence)
        
        self.blurSlider.valueChanged.connect(self.blurValue.setValue)
        self.blurValue.valueChanged.connect(self.blurSlider.setValue)
        self.blurToggle.toggled.connect(self.blurValue.setEnabled)
        self.blurToggle.toggled.connect(self.blurSlider.setEnabled)
        self.blurToggle.toggled.connect(self.onBlur)
        
        self.dilationToggle.toggled.connect(self.dilationSlider.setEnabled)
        self.dilationToggle.toggled.connect(self.onDilate)
        self.dilationValue.valueChanged.connect(self.dilationSlider.setValue)
        self.dilationSlider.valueChanged.connect(self.dilationValue.setValue)
        self.dilationToggle.toggled.connect(self.dilationValue.setEnabled)
        
        self.medianToggle.toggled.connect(self.medianSlider.setEnabled)
        self.medianToggle.toggled.connect(self.onMedian)
        self.medianValue.valueChanged.connect(self.medianSlider.setValue)
        self.medianSlider.valueChanged.connect(self.medianValue.setValue)
        self.medianToggle.toggled.connect(self.medianValue.setEnabled)

        self.contourSlider.valueChanged.connect(self.contourValue.setValue)
        self.contourValue.valueChanged.connect(self.contourSlider.setValue)
        self.contourToggle.toggled.connect(self.contourValue.setEnabled)
        self.contourToggle.toggled.connect(self.contourSlider.setEnabled)
        self.contourToggle.toggled.connect(self.contourFunction) 
        
        self.contrastSlider.valueChanged.connect(self.contrastValue.setValue)
        self.contrastValue.valueChanged.connect(self.contrastSlider.setValue)
        self.contrastToggle.toggled.connect(self.contrastValue.setEnabled)
        self.contrastToggle.toggled.connect(self.contrastSlider.setEnabled)
        self.contrastToggle.toggled.connect(self.contrastFunction)    

        self.thresholdSlider.valueChanged.connect(self.thresholdValue.setValue)
        self.thresholdValue.valueChanged.connect(self.thresholdSlider.setValue)
        self.thresholdToggle.toggled.connect(self.thresholdValue.setEnabled)
        self.thresholdToggle.toggled.connect(self.thresholdSlider.setEnabled)     
        self.thresholdToggle.toggled.connect(self.thresholdFunction)    
        
        self.medianValue.valueChanged.connect(self.onMedian)
        self.blurValue.valueChanged.connect(self.onBlur)
        self.dilationValue.valueChanged.connect(self.onDilate)
   
        # initialize values
        self.thresholdVal = 0
        self.contrastVal = 1
        self.contourVal = 1

        # initialize video display
        self.videos = []
        self.testButton.clicked.connect(self.addVideos)
        self.testButton.hide() # test button to add blank videos
        self.emptyVideo = QPixmap(u"background.jpg") # blank video background
        self.emptyTruth = True
        self.numColumns = 2
        self.dim = [640, 480]
        self.numVideos = 1
        self.countPlaceholder=0
        self.COUNTADDVIDEO = 0 # testing  count variables
        self.COUNTCAPTURE = 0
        self.COUNTPARSE = 0
        self._running = True
        self.backSubsMOG2={} # background subtraction
        self.backSubsKNN={}
        self.subBackVal = 0
        self.blurVal=0
        self.dilateVal=0
        self.medianVal=0
        self.consecutiveFrames=12 # for frame diff
        self.consecutiveFramesValue.setValue(12)
        self.frameDifVal=30
        self.frameDiffValue.setValue(30)
        self.resetFrames=False
        self.bb = True # bounding boxes
        self.points = False # points
        self.persistence = 30

        # set empty placeholder video
        self.video_1.setIcon(QtGui.QIcon(u"background.jpg"))
        self.video_1.setMaximumSize(self.dim[0], self.dim[1])
        self.video_1.setIconSize(QtCore.QSize(self.dim[0]*.99, self.dim[1]*.99))
        self.video_1.setCheckable(True)
        self.video_1.setStyleSheet("border: none; background: transparent;")

        # update plots
        self.timer = QTimer()
        self.timer.timeout.connect(self.updatePlots)
        self.timer.start(33)

        # data collection
        self.data={}
        self.dataTemp={}
        self.tracker = cv2.TrackerKCF_create()
        self.fps = 30
        self.pixToum = 12.6

        # data view
        self.summaryPlot.showGrid(x=True, y=True)
        self.summaryPlot.setLabel("left", "Velocity (um/s)")
        self.summaryPlot.setLabel("bottom", "Time (s)")
        self.summaryPlot.setLimits(xMin=0, xMax=60, yMin=-5, yMax=5)
        self.summaryPlot.setXRange(0, 30)
        self.summaryPlot.setYRange(-5, 5)
        self.summaryData=defaultdict(list)
        self.tempData=defaultdict(list)
        self.rectBuffer={}
        self.velocities=[] 
        self.videoNames=[]
        self.curves={}
        self.colors={}
        self.previousCRFunction=''
        self.npmData={}
        self.show
    
    def onCollateData(self):
        self.NPMPlot.show()
    
    def updatePlots(self):
        for item in self.summaryPlot.items(): # return all labels
            if isinstance(item, pg.TextItem):
                self.summaryPlot.removeItem(item) # clear each label
        for video in self.videoNames:
            try:
                self.summaryData[video] # check if video exists
            except Exception as e:
                print(f'Error reading data for {e}', end='\r')
                return
            if not self.summaryData[video] == self.tempData[video] and self.record.isChecked(): # check if record button is pressed and new data is different from previous frame
                duplicate = set()
                unique = []
                try:
                    for coord in self.summaryData[video]:
                        if coord[0] not in duplicate: # remove duplicate; can't have 2 different velocities for a given time
                            unique.append(coord)
                            duplicate.add(coord[0])
                    t, v = zip(*unique)
                    if video not in self.curves: # initialize curve for each video
                        pen = pg.mkPen(color=self.colors[video])
                        self.curves[video] = self.summaryPlot.plot(t, v, pen=pen, symbol=None, name='Median Velocity')
                    else: # update instead of clearing and replotting each curve
                        self.curves[video].setData(t, v)
                    label = pg.TextItem(text=f'({round(v[-1],2)}, {round(t[-1],2)})', anchor=(0.5, 1))
                    label.setPos(t[-1], v[-1])
                    self.summaryPlot.addItem(label) # add velocity label
                    self.tempData[video]=self.summaryData[video].copy() # maintain current frame data to compare to next frame

                    '''
                    NPM Data Extraction
                    '''

                    voltage = float(video.split('_')[1].split('V')[0])
                    filteredVelocity, error = self.dataStatistics('IQR', v)
                    meanVelocity = np.mean(filteredVelocity)
                    self.npmData[voltage]=(meanVelocity, error)
                except Exception as e:
                    err = traceback.format_exc()
                    print(err)
                    pass
        self.summaryPlot.update() # update GUI
        if self.NPMPlot.isVisible():
            data=[]
            for voltage in self.npmData:
                velocity = self.npmData[voltage][0]
                error = self.npmData[voltage][1]
                data.append((voltage, velocity, error))
            data=sorted(data)
            self.NPMPlot.updatePlot(data)

    def dataStatistics(self, method, data):
        try:
            if method == 'IQR':
                data=np.array(data)
                # Calculate Q1 (25th percentile) and Q3 (75th percentile)
                Q1 = np.percentile(data, 25)
                Q3 = np.percentile(data, 75)
                
                # Calculate IQR
                IQR = Q3 - Q1
                
                # Define lower and upper bounds for outliers
                lowerBound = Q1 - 1.5 * IQR
                upperBound = Q3 + 1.5 * IQR
                
                # Filter out outliers
                filteredData = data[(data >= lowerBound) & (data <= upperBound)]
                error = IQR / 2
        
                return filteredData, error
            else:
                print(f'{method} not a statistical method!')
                pass
        except Exception as e:
            err = traceback.format_exc()
            print(err)

    def onPersistence(self):
        self.persistence = self.contDetectValue.value()
    
    def onRecord(self, checked):
        global data
        global videoData
        try:
            data = pd.DataFrame(self.summaryData)
            videoData = pd.DataFrame(self.data)
        except:
            pass

        QTimer.singleShot(100, self.clearPlots)

    def clearPlots(self):
        if self.record.isChecked():
            self.summaryPlot.clear()
            self.tempData.clear()
            self.curves.clear()
            for video in self.videoNames:
                try:
                    self.data[video].clear()
                    self.summaryData[video].clear()
                    self.printNewLine(f'Cleared plot for: {video}')
                except:
                    pass
            self.summaryPlot.update()
                    
    def onPoints(self):
        self.actionBounding_Boxes_2.setChecked(False)
        self.bb = False
        self.points = True
    
    def onBB(self):
        self.actionPoints_2.setChecked(False)
        self.bb = True
        self.points = False
            
    def checkCounts(self):
        self.printCarriageReturn(f'{(self.COUNTPARSE, self.COUNTADDVIDEO, self.COUNTCAPTURE)}')

    def onScaleBar(self):
        self.scaleBarOn+=1
    
    def onClear(self):
        clearVideos(self.videoLayout) # clear videos
        
        # Perform cleanup tasks
        self._running=False
        for connection in self.videoConnections:
            connection.signals.result.disconnect(self.onFrameReady)
            connection.stop()

        # Clean up video connections
        self.videoConnections.clear()
        self.threadPool.waitForDone()

        # create blank object
        tempVideo = QPushButton('')
        tempVideo.setObjectName('1_blank')
        tempVideo.setCheckable(True)
        tempVideo.setMaximumSize(self.dim[0], self.dim[1])
        tempVideo.setIcon(QtGui.QIcon(u"background.jpg"))
        tempVideo.setIconSize(QtCore.QSize(self.dim[0]*.99, self.dim[1]*.99))
        tempVideo.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)

        self.numColumns = 2
        self.countPlaceholder=0
        self.numVideos=1
        self.videos.clear()
        self.emptyTruth = True
        self._running=True
        self.videoNames.clear()
        self.videoFiles.clear()
        self.summaryData.clear()
        self.videoLayout.addWidget(tempVideo, 0, 0)    

    def cropFrame(self, frame, targetSize):
            # check if frame needs to be cropped
            oldH, oldW = frame.shape[:2]
            targetH, targetW = targetSize
            oldAR = oldW/oldH
            newAR = targetW/targetH

            # stretch image to fit while maintaining aspect ratio
            if oldAR > newAR: # original is wider, scale by height
                newH = targetH
                newW = int(targetH*oldAR)
            else: # original is taller, scale by width
                newW = targetW
                newH = int(targetW/oldAR)

            # resize while maintaining aspect ration
            resizedFrame = cv2.resize(frame, (newW, newH), interpolation=cv2.INTER_AREA) # resize to PyQt frame

            # calculate cropping coordinates
            startX = (newH-targetW) // 2
            startY = (newH - targetH) // 2

            # crop image
            croppedFrame = resizedFrame[startX:startX + targetW, startY:startY + targetH]

            return croppedFrame
    
    def captureVideos(self, videoPath, videoName, resultCallback):
        cap = cv2.VideoCapture(videoPath)
        truth = True
        checked = False
        frames=[]
        evenFrame=None
        oddFrame=None
        temp=0
        detectedContours=None
        n=0
        medianFrames=[]
        timer=None
        runOnce=True
        stopData=False
        collectData={}
        
        if not cap.isOpened():
            msg = f"Error: Could not open video {videoName}.\n"
            self.printNewLine(msg)
            return   
        
        while self._running:
            ret, frame = cap.read()
            timer = cv2.getTickCount()
            
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to the first frame
                if self.record.isChecked() and not runOnce and not stopData:
                    stopData=True
                    self.printNewLine(f'Switching to collected data for {videoName}. . .')
                n=0
                continue  # Continue the loop to read the video from the start

            if self.record.isChecked() and runOnce:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                self.summaryData[videoName].clear()
                self.data[videoName].clear()
                collectData.clear()
                frames.clear()
                detectedContours=[]
                n=0
                runOnce=False
                stopData=False
            elif not self.record.isChecked():
                runOnce=True

            originalFrame = frame.copy()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # remove colors, most processes, such as frame differencing require this
            
            for video in self.videos:
                if video.objectName() == videoName:
                    checked = video.isChecked() # check if any video is checked/clicked on
            dark = False

            try:
                if checked or len(self.videos) == 0:
                    if self.contrastToggle.isChecked(): # contrast
                        frame = cv2.convertScaleAbs(frame, alpha=self.contrastVal, beta=0)
                        
                    if self.blurToggle.isChecked(): # gaussian blur
                        frame = cv2.GaussianBlur(frame, (self.blurVal, self.blurVal), 0)
                        
                    if self.subBackToggle.isChecked(): # subtract background
                        if self.subBackMethod.currentText() == "MOG2": # Mixture of Gaussians 2 background subtraction method
                            frame = self.backSubsMOG2[videoName].apply(frame)
                        if self.subBackMethod.currentText() == "KNN": # K Nearest Neighbors background subtraction method
                            frame = self.backSubsKNN[videoName].apply(frame)
                            
                    if self.contourToggle.isChecked(): # emboss, misnamed as "contour" when originally writing, too lazy to replace every reference
                        kernel = np.array([[2, 1, 0],[1, 0, -1],[0, -1, -2]])
                        frame = cv2.convertScaleAbs(cv2.filter2D(frame, -1, kernel)*self.contourVal)
                        
                    if self.thresholdToggle.isChecked(): # threshold
                        _, frame = cv2.threshold(frame, self.thresholdVal, 255, cv2.THRESH_BINARY)

                    if self.dilationToggle.isChecked(): # dilation for emboss
                        frame = cv2.dilate(frame, None, iterations=self.dilateVal)
                    
                    medianFrames.append(frame)
                    if len(medianFrames) >= self.medianVal:
                        medianFrames = medianFrames[-self.medianVal:]
                    if self.medianToggle.isChecked() and self.medianVal > 0: # median frame filtering, helps remove flicker after applying all filters
                        # Stack frames and apply median filter
                        stack = np.stack(medianFrames, axis=2)
                        frame = np.average(stack, axis=2).astype(np.uint8)

                    if self.thresholdToggle.isChecked(): # threshold
                        _, frame = cv2.threshold(frame, self.thresholdVal, 255, cv2.THRESH_BINARY)
                        
                    elif len(self.videos) != 0:
                        dark = True
                    if any([self.contrastToggle.isChecked(),
                                                          self.blurToggle.isChecked(),
                                                          self.dilationToggle.isChecked(),
                                                          self.subBackToggle.isChecked(),
                                                          self.contourToggle.isChecked(),
                                                          self.thresholdToggle.isChecked(),
                                                          self.frameDiffToggle.isChecked()]): # don't darken if only one video is shown or if any process is toggled
                        dark = False

                    if not stopData:
                        frameCount=cap.get(cv2.CAP_PROP_POS_FRAMES)
                            
                        if frameCount % 2 == 0:
                            evenFrame=frame.copy()
                            temp=cap.get(cv2.CAP_PROP_POS_FRAMES)
                        elif frameCount > temp or frameCount == 1: # make sure that it takes the difference from a frame and a previous frame, not sure if necessary
                            oddFrame=frame.copy()
    
                        if evenFrame is not None and oddFrame is not None:
                            frames.append(cv2.absdiff(oddFrame, evenFrame)) # find difference between current 2 most recent frames
    
                        if len(frames) > self.consecutiveFrames:
                            frames.pop(0)
    
                        if len(frames) > self.consecutiveFrames + 1:
                            self.printNewLine(f'Cached frames "{len(frames)}" exceeds CF "{self.consecutiveFrames}" for {videoName}! Clearing. . .')
                            frames=frames[:-self.consecutiveFrames]

                    if not stopData:
                        self.rectBufferMutex.lock() # thread locking
                    try:
                        if len(frames) == self.consecutiveFrames:
                            if self.frameDiffToggle.isChecked():
                                try:
                                    if not stopData:
                                        detectedContours, trackedContours, trackedObjects = self.frameDifference(frames, videoName, frameCount, n)
                                        collectData[n]=(detectedContours, trackedContours, trackedObjects)
                                    else:
                                        detectedContours, trackedContours, trackedObjects = collectData[n]
                                except Exception as e:
                                    err = traceback.format_exc()
                                    self.printNewLine(f'Error detecting contours for {videoName}: {err}')
                                n+=1
                        try:
                            if detectedContours is not None and self.frameDiffToggle.isChecked(): # have bounding boxes persist
                                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
                                if self.actionOriginal_Frame.isChecked():
                                    frame = originalFrame
                                if self.bb:
                                    if self.record.isChecked() and trackedContours and trackedObjects:
                                        for trackedContour in trackedContours:
                                            try:
                                                cv2.rectangle(frame, trackedContour[0], trackedContour[1], self.colors[videoName][::-1], thickness=2)
                                            except Exception as e:
                                                self.printNewLine(f'Error drawing tracking rects for {videoName}: {e}')
                                        for trackedObject in trackedObjects:
                                            try:
                                                cv2.putText(frame, f'{trackedObject[0]}, {trackedObject[2]} um/s', trackedObject[1][0],
                                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors[videoName][::-1], 1)  
                                            except Exception as e:
                                                self.printNewLine(f'Error drawing text for {videoName}: {e}')
                                    else:
                                        try:
                                            if self.record.isChecked():
                                                color=tuple(255 - component for component in self.colors[videoName][::-1])
                                            else:
                                                color = self.colors[videoName][::-1]
                                            for rect in detectedContours[0]:
                                                if len(rect) == 2 and len(rect[0]) == 2 and len(rect[1]) == 2:
                                                    cv2.rectangle(frame, rect[0], rect[1], color, thickness=2)
                                                else:
                                                    self.printNewLine(f'Improper shape for {videoName}: {rect}')
                                        except Exception as e:
                                            pass
                                else:
                                    try:
                                        if self.record.isChecked() and trackedObjects:
                                            for trackedObject in trackedObjects:
                                                cv2.circle(frame, trackedObject[1][1], radius=2, color=self.colors[videoName][::-1], thickness=-1)
                                                cv2.putText(frame, f'{trackedObject[0]}, {trackedObject[2]} um/s', trackedObject[1][1],
                                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors[videoName][::-1], 1)  
                                        else:
                                            for point in detectedContours[1]:
                                                cv2.circle(frame, point, radius=2, color=self.colors[videoName][::-1], thickness=-1)
                                    except Exception as e:
                                        self.printNewLine(f'Error drawing points for {videoName}: {e}')
                        except Exception as e:
                            err = traceback.format_exc()
                            self.printNewLine(f'Error drawing contours for {videoName}: {err}')
                    finally:
                        if not stopData:
                            self.rectBufferMutex.unlock()
            except Exception as e:
                self.printNewLine(e)

            if self.actionOriginal_Frame.isChecked() and not self.frameDiffToggle.isChecked():
                frame = originalFrame
            
            if self.scaleBarOn % 2 == 0: # add scale bar
                scaleBar(frame, scaleFactor=12.6, scaleLength=30, divisions = 6, posX = 5, posY = 5, fontScale = 1.5, thickness = 1, border = 1, barHeight = 30)
                
            fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
            if self.actionFPS.isChecked():
                msgFPS = f'FPS : {str(int(fps))}'
                pos=(5, 5)
                fontScale = 1.5
                thickness = 2
                rectFPS, posFPS = self.textBackground(msgFPS, fontScale, thickness, pos)
                cv2.rectangle(frame, rectFPS[0], rectFPS[1], color=(255, 255, 255), thickness=-1)
                cv2.putText(frame, msgFPS, posFPS, cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0,0,0), thickness, lineType=cv2.LINE_AA)

            if self.actionDetails.isChecked():
                height, width = frame.shape[:2]
                if self.actionFPS.isChecked():
                    pos=(5, rectFPS[1][1]+5)
                else:
                    pos=(5, 5)
                fontScale = 1.5
                thickness = 2
                msgDetails=f'{videoName.split("_")[1]}, {width}x{height}'
                rectDetails, posDetails = self.textBackground(msgDetails, fontScale, thickness, pos)
                cv2.rectangle(frame, rectDetails[0], rectDetails[1], color=(255, 255, 255), thickness=-1)
                cv2.putText(frame, msgDetails, posDetails, cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0,0,0), thickness, lineType=cv2.LINE_AA)
            
            if dark:
                frame = np.clip(frame*0.5, 0, 255).astype(np.uint8) # darken videos when selected
            pixmap = cv2FrameToPixmap(frame)  # convert cv2 frame to PyQt pixmap
            resultCallback.emit((pixmap, videoName)) # send frame from QRunnable thread to PyQt widget
            sleep(1/cap.get(cv2.CAP_PROP_FPS))  # adjust so that each frame is sent at the frame rate of the video, not just how fast it can process
            
            if truth:
                self.COUNTCAPTURE+=1
                # return upon successful first reading video, basically just a check that video was indeed read initially
                msg = f"Captured frame from {videoName} with dimensions: {frame.shape if frame is not None else 'None'}\n"
                self.printNewLine(msg)
                truth = False

        cap.release() # close video

    def textBackground(self, text, fontScale, thickness, pos):
        (textWidth, textHeight), baseline = cv2.getTextSize(str(text), cv2.FONT_HERSHEY_SIMPLEX, fontScale, thickness)
        rect = [(pos[0], pos[1]), (pos[0] + textWidth, pos[1] + textHeight+baseline)]
        textPos = (pos[0], pos[1] + textHeight + baseline // 2)
        
        return (rect, textPos)

    def rectBorder(self, start, end, borderL, borderR, borderU, borderD): # return border positions for any given rect
        newStart = (start[0]-borderL, start[1]-borderU)
        newEnd = (end[0] + borderR, end[1]+borderD)
        return [newStart, newEnd]

    def returnVideos(self, layout):
        rows = layout.rowCount()
        cols = layout.columnCount()
        videos=[]
        for row in range(rows): 
            for col in range(cols):
                widget = layout.itemAtPosition(row, col)
                if widget is not None:
                    videos.append(widget.widget())
        return videos

    def onFrameReady(self, result):
        if result is not None:
            pixmap = result[0]
            name = result[1]
            videos = self.returnVideos(self.videoLayout)
            for video in videos:
                try:
                    if video.objectName() == name:
                        video.setIcon(QtGui.QIcon(pixmap))
                except Exception as e:
                    msg = f'No match for {video.objectName()}: {e}'
                    self.printNewLine(msg)
        else:
            pass
            
    def addVideos(self, name='BLANK', path=None): 
        if self.emptyTruth:
            self.videoLayout.itemAtPosition(0, 0).widget().setObjectName(f"{self.numVideos}_{name}")
            self.emptyTruth = False
        elif not self.numVideos == self.numColumns**2: # stop at max
            self.countPlaceholder+=1  
            self.numVideos+=1
            
            # Calculate the row and column for the widget (I have no idea why I need to add 1 but it works)
            row = self.countPlaceholder // (self.numColumns + 1) # Integer division for row index
            column = self.countPlaceholder % (self.numColumns + 1) # Modulo operation for column index
            if column == 1: # no clue why this works
                column = 2
                self.countPlaceholder+=1                
            
            tempVideo = QPushButton('')
            tempVideo.setObjectName(f"{self.numVideos}_{name}")
            tempVideo.setCheckable(True)
            tempVideo.setIcon(QtGui.QIcon(u"background.jpg"))
            tempVideo.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
            self.videoLayout.addWidget(tempVideo, row, column)  # Add to the layout
    
            rows = self.videoLayout.rowCount()
            cols = self.videoLayout.columnCount()
            width = self.dim[0]/rows
            height = self.dim[1]/(cols-1) # no clue
            self.videos = resizeVideos(rows, cols, width, height, self.videoLayout) # resize videos to fit
        else:
            self.countPlaceholder=0
            clearVideos(self.videoLayout)
            self.numColumns+=1
            self.numVideos+=1
            for video in self.videos:
                # Calculate the row and column for the widget
                row = self.countPlaceholder // (self.numColumns + 1) # Integer division for row index
                column = self.countPlaceholder % (self.numColumns + 1) # Modulo operation for column index
                if column == 1:
                    column = 2
                    self.countPlaceholder+=1   
                self.videoLayout.addWidget(video, row, column)  # Add to the layout
                self.countPlaceholder+=1

                        
            row = self.countPlaceholder // (self.numColumns + 1) # Integer division for row index
            column = self.countPlaceholder % (self.numColumns + 1) # Modulo operation for column index
            if column == 1:
                column = 2
                self.countPlaceholder+=1   
                
            tempVideo = QPushButton('')
            tempVideo.setObjectName(f"{self.numVideos}_{name}")
            tempVideo.setCheckable(True)
            tempVideo.setIcon(QtGui.QIcon(u"background.jpg"))
            tempVideo.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
            self.videoLayout.addWidget(tempVideo, row, column)  # Add to the layout
            
            rows = self.videoLayout.rowCount()
            cols = self.videoLayout.columnCount()
            width = self.dim[0]/rows
            height = self.dim[1]/(cols-1) # no clue
            self.videos = resizeVideos(rows, cols, width, height, self.videoLayout) # resize videos to fit
        if path:
            videoName = f"{self.numVideos}_{name}"
            self.colors[videoName]=tuple(np.random.randint(50, 200, size=3).tolist())
            self.videoNames.append(videoName)
            self.data[videoName]={}
            self.dataTemp[videoName]={}
            self.COUNTADDVIDEO+=1
            self.printNewLine(f'Added video: {videoName}')
            playVideo = VideoWorker(self.mutex, self.captureVideos, path, videoName)
            playVideo.signals.result.connect(self.onFrameReady)
            self.videoConnections.append(playVideo)
            self.threadPool.start(playVideo)  # Start the worker
            self.backSubsMOG2[videoName] = cv2.createBackgroundSubtractorMOG2(history=450, varThreshold=16+48*self.subBackVal)
            self.backSubsKNN[videoName] = cv2.createBackgroundSubtractorKNN(dist2Threshold=100+500*self.subBackVal, history=500, detectShadows=True)
    
    def printNewLine(self, msg):
        # scroll to the bottom
        msg = str(msg)
        self.printOutput.append(msg)
        print(msg)
                
    def printCarriageReturn(self, msg):
        msg = str(msg)
        currentText = self.printOutput.toPlainText() # get current text
        lines = currentText.splitlines() # separate into lines at \n
        callerFunctionName = inspect.stack()[1].function

        # overwite last message
        if callerFunctionName == self.previousCRFunction:
            lines[-1] = msg
        else:
            lines.append(msg)
        

        # join lines back and update widget
        self.printOutput.setPlainText('\n'.join(lines))

        # scroll to the bottom
        self.printOutput.verticalScrollBar().setValue(self.printOutput.verticalScrollBar().maximum())

        print(msg.ljust(200), end='\r')
        self.previousCRFunction=callerFunctionName
    
    def thresholdFunction(self, *args):
        self.thresholdVal = (self.thresholdValue.value()/100)*255
        msg = f'Setting threshold to: {int(100*self.thresholdVal/255)}%'
        if self.initialized:
            if type(args[0]) == bool: # check for "toggled" state 
                if args[0]:
                    self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
            else:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed

    def contrastFunction(self, *args):
        self.contrastVal = (self.contrastValue.value()/100)+1 # values greater than one increase intensity, less than decrease
        msg = f'Setting contrast to: {int(100*self.contrastVal/2)}%'
        if self.initialized:
            if type(args[0]) == bool: # check for "toggled" state 
                if args[0]:
                    self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
            else:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed

    def contourFunction(self, *args):
        self.contourVal = (self.contourValue.value()/100)+1 # values greater than one increase intensity, less than decrease
        msg = f'Setting emboss to: {int(self.contourValue.value())}%'
        if self.initialized:
            if type(args[0]) == bool: # check for "toggled" state 
                if args[0]:
                    self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
            else:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed

    def onBlur(self, *args):
        def nearestOdd(n):
            # If n is odd, return it, otherwise add 1 to make it odd
            return n if n % 2 == 1 else n + 1
        self.blurVal=nearestOdd(int(31*self.blurValue.value()/100)) # 31 is the reccomended max given by ChatGPT
        msg=f'Setting blur to {int(100*self.blurVal/31)}%'
        if self.initialized:
            if type(args[0]) == bool: # check for "toggled" state 
                if args[0]:
                    self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
            else:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed

    def onDilate(self, *args):
        self.dilateVal=int(self.dilationValue.value())
        msg=f'Setting dilation to {int(100*self.dilateVal/5)}%'
        if self.initialized:
            if type(args[0]) == bool: # check for "toggled" state 
                if args[0]:
                    self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
            else:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed

    def onMedian(self, *args):
        self.medianVal=int(self.medianValue.value())
        msg=f'Averaging over {self.medianVal} frames'
        if self.initialized:
            if type(args[0]) == bool: # check for "toggled" state 
                if args[0]:
                    self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
            else:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed
    
    def onFrameDifferencing(self, *args): # allow for multiple types of parameters
        self.frameDifVal = self.frameDiffValue.value()
        msg = f'Frame Differencing with Sensitivity: {self.frameDifVal}%'
        if self.initialized:
            if type(args[0]) == bool: # check for "toggled" state 
                if args[0]:
                    self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
            else:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed
        if not self.initialized:
            self.initialized=True

    def onConsecutiveFramesValue(self):
        self.consecutiveFrames = self.consecutiveFramesValue.value()
        
    def frameDifference(self, frames, name, frameCount, n):
        if self.resetFrames:
            self.printCarriageReturn(f'Waiting on frames for {name}. . .')
            return None
        else:
            rectangles=[]
            points=[]
            rectanglesCleaned=[]
            if not isinstance(frames, list):
                self.printNewLine(f'{frames} is not a list of frames for {name}!')
                return None
            else:
                # sum average difference over given set of frames, helps with smoothing out consistent motion
                sumFrames = cv2.convertScaleAbs(sum(frames))
                # find the contours around the white segmented areas
                contours, hierarchy = cv2.findContours(sumFrames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    # continue through the loop if contour area is less than some given sensitivity...
                    # ... helps in removing noise detection
                    if cv2.contourArea(contour) < self.frameDifVal:
                        continue
                    # get the xmin, ymin, width, and height coordinates from the contours
                    (x, y, w, h) = cv2.boundingRect(contour)
                    # save the bounding boxes
                    rectangles.append([(x, y),(x+w, y+h)])
            
            # collect two boxes
            tempRectangles = rectangles[:]
            while tempRectangles:
                try:
                    rectA = tempRectangles.pop(0) # Take the first rectangle
                    merged = False # Flag to track if rectA has been merged
                except Exception as e:
                    self.printNewLine(e)

                while merged:  # Keep merging as long as there's an overlap
                    merged = False  # Reset the merged flag for this iteration
                
                    for rectB in tempRectangles[:]: # Use a copy of the list for iteration
                        try:
                            if self.isOverlapping(rectA, rectB):
                                rectanglesCleaned.append(self.mergeRects(rectA, rectB)) # Merge rectA with rectB
                                tempRectangles.remove(rectB) # Remove rectB from tempRectangles
                                merged = True # Mark that a merge happened
                        except Exception as e:
                            self.printNewLine(f'Error merging bounding boxes for {name}: {e}')
                            
                rectanglesCleaned.append(rectA)  # Add the merged rectA to the cleaned list    

            for rectangle in rectanglesCleaned: # find points
                points.append(self.findCenter(rectangle))

            # newRect=[]
            # for point in points:
            #     # Width and height of the rectangle
            #     width = 10
            #     height = 10
                
            #     # Calculate the top-left corner of the rectangle
            #     top_left = (int(point[0] - width / 2), int(point[1] - height / 2))
            #     # Calculate the bottom-right corner of the rectangle
            #     bottom_right = (int(point[0] + width / 2), int(point[1] + height / 2))
            #     newRect.append((top_left, bottom_right))
            # rectanglesCleaned=newRect
     
            frameVelocities = []
            trackedRectangles= []
            trackedObjects= []
            timeStamp = frameCount/self.fps
            if not self.record.isChecked(): # do nothing if record not pressed
                self.data[name]={}
                self.dataTemp[name]={}
                self.rectBuffer[name]={}
                return (rectanglesCleaned, points), False, False
            elif rectanglesCleaned:
                self.dataTemp[name][n]={}
                if not self.data[name]: # assign initial points
                    self.printNewLine(f'Initializing tracking for {name}. . .')
                    i=0
                    for rectangle in rectanglesCleaned: # save all detected rects in first frame to compare to for 2nd frame
                        self.data[name][i]={timeStamp : rectangle}
                        self.dataTemp[name][n][i]={timeStamp : rectangle} # used to return specifically previous frame rectangle information
                        self.rectBuffer[name][i]=[]
                        i+=1
                    return (rectanglesCleaned, points), False, False
                else:
                    oldRectangleObjects = list(self.dataTemp[name][n-1].keys())
                    j=max(oldRectangleObjects) # set j equal to last detected object
                    oldTimeStamp = list(self.dataTemp[name][n-1][oldRectangleObjects[0]].keys())[0]

                    for newRectangle in rectanglesCleaned:
                        match=False # declare tracked object boolean
                        for oldRectangleObject in oldRectangleObjects:
                            oldRectangle = self.dataTemp[name][n-1][oldRectangleObject][oldTimeStamp] # return each rect from the previous frame
                            if self.isOverlapping(oldRectangle, newRectangle): # check if overlapping
                                match=True
                                centerDiff = np.diff([self.findCenter(oldRectangle),self.findCenter(newRectangle)], axis=0) # return difference in pixels between centers
                                #centerDiffY = np.diff([self.findCenter(oldRectangle)[1],self.findCenter(newRectangle)[1]], axis=0)/self.pixToum # y-vector diff
                                #centerDiffX = np.diff([self.findCenter(oldRectangle)[0],self.findCenter(newRectangle)[0]], axis=0)/self.pixToum # x-vector diff
                                distance = np.linalg.norm(centerDiff)/self.pixToum # calculate euclidean distance between center points
                                direction = np.sign(centerDiff[0])[1]*-1 # calculate if vector is pointing up or down
                                timeDif = timeStamp-oldTimeStamp # calculate time delta
                                velocity = distance/timeDif*direction # calculate instantaneous velocity
                                #velocity = centerDiffY/timeDif # velocity of Y or X
                                labelPos = (newRectangle[1][0],newRectangle[0][1]) # (x2, y1)
                                trackedRectangles.append(newRectangle) # append tracked rectangles to display

                                self.data[name][oldRectangleObject].update({timeStamp : newRectangle}) # globalize tracked rectangle
                                self.dataTemp[name][n][oldRectangleObject]={timeStamp : newRectangle} # replace previous frame rect information

                                if oldRectangleObject not in self.rectBuffer[name]:
                                    self.rectBuffer[name][oldRectangleObject]=[]
                                self.rectBuffer[name][oldRectangleObject].append(velocity) # append velocity to buffer to determine average velocity
                                    
                                buffer = self.rectBuffer[name][oldRectangleObject]
                                appendix=[oldRectangleObject,[labelPos, self.findCenter(newRectangle)], 'nan']
                                if buffer:
                                    if len(buffer) > int(self.persistence) + 1:
                                        #print(f'buffer overflow on rect_{oldRectangleObject}, size: {len(buffer)}')
                                        self.rectBuffer[name][oldRectangleObject] = self.rectBuffer[name][oldRectangleObject][:-self.persistence] # remove any overflow from buffer
                                    if len(buffer) < int(self.persistence): # return no velocity if buffer is not filled
                                        #print(f'buffer not filled on rect_{oldRectangleObject}, size: {len(buffer)}')
                                        appendix = [oldRectangleObject,[labelPos, self.findCenter(newRectangle)],'...']
                                    if len(buffer) > int(self.persistence) and len(buffer) > 1: # remove first element if buffer is overfilled
                                        #print(f'buffer overfilled on rect_{oldRectangleObject}, size: {len(buffer)}')
                                        self.rectBuffer[name][oldRectangleObject].pop(0)
                                    if len(buffer) == int(self.persistence): # calculate average velocity if buffer is filled
                                        #print(f'buffer filled on rect_{oldRectangleObject}, size: {len(buffer)}')
                                        meanVelocity = np.mean(buffer)
                                        frameVelocities.append(meanVelocity)
                                        appendix = [oldRectangleObject,[labelPos, self.findCenter(newRectangle)], round(meanVelocity,2)]
                                trackedObjects.append(appendix)
                                break
                        if not match:
                            j+=1 # assign newly detected rect to a new object
                            self.data[name][j]={timeStamp : newRectangle}
                            self.dataTemp[name][n][j]={timeStamp : newRectangle}
                            self.rectBuffer[name][j]=[]

                if frameVelocities:
                    if np.array(frameVelocities).size == 0:
                        self.printNewLine(f'Array "frameVelocities" is empty for {name}! {frameVelocities}')
                    else:
                        self.summaryData[name].append((timeStamp, np.median(frameVelocities)))
                        #print(f'Frame: {np.median(frameVelocities)} um/s'.ljust(200), end='\r') 
                if not trackedRectangles:
                    self.printNewLine(f'Error: trackedRectangles empty for {name}! {trackedRectangles}')
                    return (rectanglesCleaned, points), False, trackedObjects
                if not trackedObjects:
                    self.printNewLine(f'Error: trackedObjects empty for {name}! {trackedObjects}')
                    return (rectanglesCleaned, points), trackedRectangles, False
                if not trackedRectangles and not trackedObjects:
                    self.printNewLine(f'Error: trackedObjects empty for {name}! {trackedObjects} trackedRectangles empty for {name}! {trackedRectangles}')
                    return (rectanglesCleaned, points), False, False
                return (rectanglesCleaned, points), trackedRectangles, trackedObjects
    
    def mergeRects(self, rectA, rectB): # merge two rectangles
        x1 = min(rectA[0], rectB[0])
        y1 = min(rectA[1], rectB[1])
        x2 = max(rectA[2], rectB[2])
        y2 = max(rectA[3], rectB[3])
        return (x1, y1, x2, y2)  
    
    def findCenter(self, rect):
        (x1, y1), (x2, y2) = rect
        width = x2-x1
        height = y2-y1
        center = (int(x1+width/2), int(y1+height/2))
        return center
    
    def findArea(self, rect):
        (x1, y1), (x2, y2) = rect
        area = (x2-x1)*(y2-y1)
        return area
    
    def isOverlapping(self, rectA, rectB):
        (x1, y1), (x2, y2) = rectA # first box (top-left and bottom-right)
        (x3, y3), (x4, y4) = rectB # second box

        # check for overlap
        if x2 <= x3 or x4 <= x1:  # One box is completely to the left of the other
            return False
        if y2 <= y3 or y4 <= y1:  # One box is completely above the other
            return False

        # If none of the above, the boxes overlap
        return True

    def subtractBackgroundFunction(self, *args):
        self.subBackVal = self.subBackValue.value()/100
        if self.subBackValue.value() != 0:
            QTimer.singleShot(250,self.onSubBack)

    def onSubBack(self):
        for video in self.returnVideos(self.videoLayout):
            try:
                if self.subBackMethod.currentText() == "MOG2":
                    self.backSubsMOG2[video.objectName()] = cv2.createBackgroundSubtractorMOG2(history=450, varThreshold=16+48*self.subBackVal)
                elif self.subBackMethod.currentText() == "KNN":
                    self.backSubsKNN[video.objectName()] = cv2.createBackgroundSubtractorKNN(dist2Threshold=100+500*self.subBackVal, history=450, detectShadows=True)
                self.printCarriageReturn(f'Subtracting background with intensity {self.subBackVal} using {self.subBackMethod.currentText()}')
            except Exception as e:
                msg=None
                self.printCarriageReturn(f'Failed to subtract background for {video.objectName()}: {e}')

    def openFile(self):
        options = QFileDialog.Options()
        files, _ = QFileDialog.getOpenFileNames(self, "Open Files", "", "Video Files (*.avi;*.mp4;*.mov;*.mkv;*.wmv;*.flv;*.mpeg;*.mpg)", options=options)
        
        if files:
            for file in files:
                self.parseFiles(file)

    def exportVideo(self):
        options = QFileDialog.Options()
        fileName, _ = QFileDialog.getSaveFileName(self,
                                                  "Save CSV/Excel File",
                                                  "",
                                                  "CSV Files (*.csv);;Excel Files (*.xlsx)",
                                                  options=options)
        
        if fileName:
            print(f"File saved as: {fileName}")
            # Now you can use 'file_name' to save your file, e.g., writing or copying data to this path
    
    def parseFiles(self, file):
        accepted = False
        fileType = file.split(".")[-1] # return file extension
        for videoType in videoFormats:
            if fileType == videoType:
                accepted = True # return true if file is a video
                break
        if not accepted:
            msg = f'File type: ".{fileType}" not accepted. \nAllowed file types:  ".avi," ".mp4", ".mov", ".mkv", ".wmv", ".flv", ".mpeg", ".mpg"\n'
            self.printNewLine(msg)
            return
        else:
            for videoFile in self.videoFiles:
                if file == videoFile:
                    msg = f'File "{file}" already added!'
                    self.printNewLine(msg)
                    accepted = False
                    return
        if accepted:
            msg = f'File opened: {file}'
            fileName = file.split(r'/')[-1]
            self.printNewLine(msg)
            try:
                self.addVideos(fileName, file)
                self.videoFiles.append(file)
                self.COUNTPARSE+=1
            except Exception as e:
                msg = f'Failed to add file: {file} {e}'
                self.printNewLine(msg)        
    
    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls(): # check if dragged item has file location/url
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().hasUrls():
            Qfiles = event.mimeData().urls()
            files = [Qfile.toLocalFile() for Qfile in Qfiles] # convert qt-type to file path string
            for file in files:
                self.parseFiles(file)
            event.accept()
        else:
            event.ignore()
    
    def closeEvent(self, event):
        try:
            self._running=False
            # Perform cleanup tasks
            for connection in self.videoConnections:
                connection.signals.result.disconnect(self.onFrameReady)
                connection.stop()

            # Clean up video connections
            self.videoConnections.clear()
            self.threadPool.waitForDone()

            # Perform any other necessary cleanup tasks
            event.accept()  # Accept the event to close the window
            
            super().closeEvent(event)
        except Exception as e:
            msg = f"Error during close event: {e}"
            self.printNewLine(msg)
            event.ignore()  # Optionally ignore the event if cleanup fails
        print('\nExited')

if not QtWidgets.QApplication.instance():
    app = QtWidgets.QApplication(sys.argv)
else:
    app = QtWidgets.QApplication.instance()

if __name__ == '__main__':
    window = MainWindow()
    app.setStyle('Windows')
    window.show()
    print('Running\n')
    app.exec()

Running

File opened: D:/NPM/09-19-2024/DC/5V 30 sec_original.avi
Added video: 1_5V 30 sec_original.avi
Captured frame from 1_5V 30 sec_original.avi with dimensions: (1024, 1280)

Initializing tracking for 1_5V 30 sec_original.avi. . .                                                                                                                                                 
Cleared plot for: 1_5V 30 sec_original.avi
Initializing tracking for 1_5V 30 sec_original.avi. . .
Switching to collected data for 1_5V 30 sec_original.avi. . .
Error detecting contours for 1_5V 30 sec_original.avi: Traceback (most recent call last):
  File "C:\Users\Nolan\AppData\Local\Temp\ipykernel_9856\2452197180.py", line 652, in captureVideos
    detectedContours, trackedContours, trackedObjects = collectData[n]
                                                        ~~~~~~~~~~~^^^
KeyError: 1021

Error detecting contours for 1_5V 30 sec_original.avi: Traceback (most recent call last):
  File "C:\Users\Nol

In [14]:
import time as t
import re
fps = 30
pixToum = 12.6

startTime = t.time()
# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
flickerVelocities=[]
for video in data: # iterate each video
    for flicker in range(2, 100): # check for variance across longer tracked wires
        wires={}
        for index, wireObject in data[video].items(): # find each wire object
            if len(wireObject) >= flicker: # clean out flickering
                wireInfo = sorted(
                    [(int(frame.split('_')[1].split('.')[0]) / fps, # extract frame number and convert to time stamp
                      wireObject[frame]) # return positions
                     for frame in wireObject.keys()],
                    key=lambda x: x[0] # return wire index
                )
                wires[index]=wireInfo
        
        wireVelocities={}
        
        for wire, wireInfo in wires.items(): # convert time and coordinates into velocities
            times = np.array([info[0] for info in wireInfo])
            positions = np.array([info[1] for info in wireInfo]) / pixToum # convert pixel to um
            velocities=[]
            
            timeDif = np.diff(times) # calculate time differences
            
            posDif = np.linalg.norm(np.diff(positions, axis=0), axis=1) # calculate position differences
    
            velocities = posDif/timeDif
            wireVelocities[wire] = np.mean(velocities)
            
        
        avgVelocities = np.array(list(wireVelocities.values())) # aggregate velocities across wires
        if avgVelocities.size == 0: # if no valid wires, break
                break
    
        avgVelocity = np.mean(avgVelocities)
        flickerVelocities.append(avgVelocity)

medianVelocity = float(round(np.median(flickerVelocities),6))
endTime = t.time()
if not np.isnan(medianVelocity):
    print(f'Median Velocity: {medianVelocity} um/s, Runtime: {round(endTime - startTime,6)} seconds', end='\r')

In [17]:
test={}
test[2]=5
test[4]=6
test[3]=2
temp=[]
for i in test:
    temp.append((i,test[i],float(np.sin(i))))
temp=sorted(temp)
x, y, z = zip(*temp)
print(x, y, z)

(2, 3, 4) (5, 2, 6) (0.9092974268256817, 0.1411200080598672, -0.7568024953079282)
